<a href="https://colab.research.google.com/github/brianellis1997/Music_Generation/blob/main/Extraction_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Symbolic Music MIDI (Parent Paper Code)
In this notebook, we will demo extracting features from the generated MIDI file produced by our parent paper.

In [ ]:
# Clone repository
!git clone https://github.com/brianellis1997/Music_Generation.git # Clone our repository
!git clone https://github.com/slSeanWU/Compose_and_Embellish.git # Parent Paper's repo
%cd Compose_and_Embellish

# Install libraries
!pip install -r requirements.txt

# Install pre-trained transformers (15 min runtime)
!pip install git+https://github.com/cifkao/fast-transformers.git@39e726864d1a279c9719d33a95868a4ea2fb5ac5
!git clone https://huggingface.co/slseanwu/compose-and-embellish-pop1k7
!pip install miditoolkit

Cloning into 'Music_Generation'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 162 (delta 79), reused 36 (delta 7), pack-reused 0
Receiving objects: 100% (162/162), 3.80 MiB | 7.48 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/Music_Generation
ERROR: Could not find a version that satisfies the requirement torch==1.6.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1)
ERROR: No matching distribution found for torch==1.6.0
  Cloning https://github.com/cifkao/fast-transformers.git (to revision 39e726864d1a279c9719d33a95868a4ea2fb5ac5) to /tmp/pip-req-build-0sa8enr2
  Running command git clone --filter=blob:none --quiet https://github.com/cifkao/fast-transformers.git /tmp/pip-req-build-0sa8enr2
  Running command git rev-parse -q --verify 'sha^39e726864d1a279c9719d33a95868a4ea2fb5ac5'
  Running command git fetch -q 

In [ ]:
# Compose

# Generating a leadsheet
!python3 stage01_compose/inference.py \
  stage01_compose/config/pop1k7_finetune.yaml \
  generation/stage01 \
  1   # Generate one leadsheet

Please input max_bars: 64
Please input temperature (randomness): 1.4
Please input top_p: 1.3
Invalid input. Please enter a valid numeric value.
Please input top_p: 2
[nucleus parameters] t = 1.4, p = 2
[info] # params: 41331059
Traceback (most recent call last):
  File "/content/Music_Generation/Music_Generation/Music_Generation/stage01_compose/inference.py", line 131, in <module>
    pretrained_dict = torch.load(config['inference_param_path'], map_location='cpu')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 986, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 435, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 416, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'compose-and-embellish-pop1k7/compose_model_lmd_pretra

In [ ]:
# Embellish

# We will embellish our generated leadsheet
!python3 stage02_embellish/inference.py \
  stage02_embellish/config/pop1k7_default.yaml \
  generation/stage01 \
  generation/stage02

In [ ]:
# installs and imports to convert MIDI into audio
!pip install pretty_midi
!wget https://www.dropbox.com/s/4x27l49kxcwamp5/GeneralUser_GS_1.471.zip
!unzip GeneralUser_GS_1.471.zip
!apt install -y fluidsynth
from pretty_midi import PrettyMIDI
from IPython.display import Audio
from scipy.io.wavfile import write
import librosa

In [ ]:
##########
# LISTEN #
##########

# render the first stage
!fluidsynth -ni GeneralUser\ GS\ 1.471/GeneralUser\ GS\ v1.471.sf2 generation/stage01/samp_01.mid -F first_stage.wav -r 44100

# render the second stage
# !fluidsynth -ni GeneralUser\ GS\ 1.471/GeneralUser\ GS\ v1.471.sf2 generation/stage02/samp_01_2stage_samp01.mid -F second_stage.wav -r 44100

# # uncomment if you want to hear the melody
# # generated in the first stage
# # hear the first stage
x,sr=librosa.load('first_stage.wav')
Audio(x,rate=sr)

# hear the second stage
# x,sr=librosa.load('second_stage.wav')
# Audio(x,rate=sr)

# Extraction
Now, we will load in our extraction methods from our extraction features notebook. We will demonstrate how we can extract the necessary features for our regression model from our generated MIDI.